# EPL DATA ANALYSIS

## Introduction 

The purpose of this notebook is to analyze the impact of covid-19 to the performance of English Premier League (EPL) teams. The dataset contained match information from : 
* 2018/2019 season (1 season before covid-19 interruption)
* 2019/2020 season (season where covid interrupted the league)
* 2020/2021 season (1 season after covid-19 interruption)

Information regarding the date of covid-19 interruption : 
* last match before covid-19 interruption = Leicester vs Aston Villa 10 March 2020 (match ID 46889)
* first match before covid-19 interruption = Aston Villa vs Sheffield United 18 June 2020 (match ID 46875)

Analysis objectives : 
* How the absence of spectators had impacted performance of EPL teams 
    * Home/Away Possession
    * Home/Away Shots on Target
    * Home/Away Shots
    * Home/Away Goals Scored per Game
    * Home/Away Win Rate
    * Home/Away Shot on Target % = Shots on Target / Shots
    * Home/Away Quantity Conversion Rate = Goals / Shots 
    * Home/Away Quality Conversion Rate  = Goals / Shots on Target
    * Home/Away Win Rate = No. of win / No. of games (Home/Away)
    * Home/Away Points per Game (PpG) = Average points/game (Home / Away)
* Relation of Ball Possession % vs Goal Conversion Rate
* Compare the performance metrics between "big-6" and "non big-6" teams before and after covid-19. (note : Big-6 = "Manchester United", "Manchester City", "Liverpool", "Arsenal", "Chelsea", "Tottenham Hotspurs")

In [1]:
import pandas as pd

#Set float number just have two decimals
pd.set_option('display.float_format','{:.2f}' .format)

In [2]:
epl_data = pd.read_csv('epl_data_cleaned.csv', index_col=[0])
epl_data

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,home_shots_on_target,away_shots_on_target,home_shots,away_shots,home_points,away_points
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,4,1,12,10,3,0
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,6,10,15,12,0,3
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,1,4,6,13,0,3
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,6,4,8,13,3,0
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,2,5,15,15,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,11,3,21,8,3,0
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,3,3,12,10,3,0
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,7,5,14,17,3,0
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,4,4,14,9,0,3


## Calculate Key Stats 

Some of the key stats required for further analysis are : 
* Shot on Target % = Shots on Target / Shots
* Quantity Conversion Rate = Goals / Shots 
* Quality Conversion Rate  = Goals / Shots on Target

In [3]:
def calculate_key_stats(df, num, div,stats_name,percentage=True):
    if percentage == True:
        #home key stats
        df['home_'+stats_name] = (df['home_'+num]/df['home_'+div]) * 100
        #away key stats
        df['away_'+stats_name] = (df['away_'+num]/df['away_'+div]) * 100
    else:
        #home key stats
        df['home_'+stats_name] = df['home_'+num]/df['home_'+div]
        #away key stats
        df['away_'+stats_name] = df['away_'+num]/df['away_'+div]
    
    #handling 0/0 value (if any)
    df = df.fillna(0, axis=1)
    
    return df

In [6]:
#calculate shot on target %
calculate_key_stats(df=epl_data, num='shots_on_target',div='shots',stats_name='SoT%')

#calculate quantity conversion rate
calculate_key_stats(df=epl_data, num='score',div='shots',stats_name='quan_CR', percentage=False)

#calculate quality conversion rate
calculate_key_stats(df=epl_data, num='score',div='shots_on_target',stats_name='qual_CR', percentage=False)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_shots,away_shots,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,12,10,3,0,33.33,10.00,0.17,0.00,0.50,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,15,12,0,3,40.00,83.33,0.00,0.17,0.00,0.20
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,6,13,0,3,16.67,30.77,0.00,0.23,0.00,0.75
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,8,13,3,0,75.00,30.77,0.25,0.08,0.33,0.25
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,15,15,0,3,13.33,33.33,0.07,0.13,0.50,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,21,8,3,0,52.38,37.50,0.24,0.00,0.45,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,12,10,3,0,25.00,30.00,0.08,0.00,0.33,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,14,17,3,0,50.00,29.41,0.21,0.00,0.43,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,14,9,0,3,28.57,44.44,0.07,0.22,0.25,0.50


## Calculate Points

In [7]:
def get_points(df):
    #home win
    df.loc[df['home_score']>df['away_score'], 'home_points'] = 3
    df.loc[df['home_score']>df['away_score'], 'away_points'] = 0

    #away win
    df.loc[df['home_score']<df['away_score'], 'home_points'] = 0
    df.loc[df['home_score']<df['away_score'], 'away_points'] = 3
    
    #draw
    df.loc[df['home_score']==df['away_score'], 'home_points'] = 1
    df.loc[df['home_score']==df['away_score'], 'away_points'] = 1
    
    return df
        
#calculate points for each game
get_points(epl_data)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_shots,away_shots,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,12,10,3,0,33.33,10.00,0.17,0.00,0.50,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,15,12,0,3,40.00,83.33,0.00,0.17,0.00,0.20
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,6,13,0,3,16.67,30.77,0.00,0.23,0.00,0.75
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,8,13,3,0,75.00,30.77,0.25,0.08,0.33,0.25
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,15,15,0,3,13.33,33.33,0.07,0.13,0.50,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,21,8,3,0,52.38,37.50,0.24,0.00,0.45,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,12,10,3,0,25.00,30.00,0.08,0.00,0.33,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,14,17,3,0,50.00,29.41,0.21,0.00,0.43,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,14,9,0,3,28.57,44.44,0.07,0.22,0.25,0.50


## Calculate Win Rate

In [8]:
def get_win_rate(df):
    #home win
    df.loc[df['home_score']>df['away_score'], 'home_win_rate'] = 1
    df.loc[df['home_score']>df['away_score'], 'away_win_rate'] = 0

    #away win
    df.loc[df['home_score']<df['away_score'], 'home_win_rate'] = 0
    df.loc[df['home_score']<df['away_score'], 'away_win_rate'] = 1
    
    #draw
    df.loc[df['home_score']==df['away_score'], 'home_win_rate'] = 0
    df.loc[df['home_score']==df['away_score'], 'away_win_rate'] = 0
    
    return df

#calculate win rate
get_win_rate(epl_data)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR,home_win_rate,away_win_rate
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,3,0,33.33,10.00,0.17,0.00,0.50,0.00,1.00,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,0,3,40.00,83.33,0.00,0.17,0.00,0.20,0.00,1.00
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,0,3,16.67,30.77,0.00,0.23,0.00,0.75,0.00,1.00
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,3,0,75.00,30.77,0.25,0.08,0.33,0.25,1.00,0.00
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,0,3,13.33,33.33,0.07,0.13,0.50,0.40,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,3,0,52.38,37.50,0.24,0.00,0.45,0.00,1.00,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,3,0,25.00,30.00,0.08,0.00,0.33,0.00,1.00,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,3,0,50.00,29.41,0.21,0.00,0.43,0.00,1.00,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,0,3,28.57,44.44,0.07,0.22,0.25,0.50,0.00,1.00


## Change Datatype

In [9]:
epl_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 0 to 1139
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   match_id              1140 non-null   int64  
 1   match_date            1140 non-null   object 
 2   matchweek             1140 non-null   object 
 3   home_team             1140 non-null   object 
 4   away_team             1140 non-null   object 
 5   season                1140 non-null   object 
 6   home_score            1140 non-null   int64  
 7   away_score            1140 non-null   int64  
 8   home_possession       1140 non-null   float64
 9   away_possession       1140 non-null   float64
 10  home_shots_on_target  1140 non-null   int64  
 11  away_shots_on_target  1140 non-null   int64  
 12  home_shots            1140 non-null   int64  
 13  away_shots            1140 non-null   int64  
 14  home_points           1140 non-null   int64  
 15  away_points          

In [10]:
string_cols = ['matchweek','home_team','away_team','season']
int_cols = ['home_score','away_score','home_shots_on_target','away_shots_on_target',
            'home_shots','away_shots','home_points','away_points']
float_cols = ['home_possession','away_possession','home_SoT%','away_SoT%','home_quan_CR','away_quan_CR' \
              ,'home_qual_CR','away_qual_CR','home_win_rate','away_win_rate']
datetime_cols = 'match_date'

#convert to corresponding datatype
epl_data[string_cols] = epl_data[string_cols].astype("string")
epl_data[int_cols] = epl_data[int_cols].astype("int")
epl_data[float_cols] = epl_data[float_cols].astype("float")

#convert object to datetime object
epl_data['match_date'] = pd.to_datetime(epl_data['match_date'], format='%Y-%m-%d')

epl_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 0 to 1139
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   match_id              1140 non-null   int64         
 1   match_date            1140 non-null   datetime64[ns]
 2   matchweek             1140 non-null   string        
 3   home_team             1140 non-null   string        
 4   away_team             1140 non-null   string        
 5   season                1140 non-null   string        
 6   home_score            1140 non-null   int32         
 7   away_score            1140 non-null   int32         
 8   home_possession       1140 non-null   float64       
 9   away_possession       1140 non-null   float64       
 10  home_shots_on_target  1140 non-null   int32         
 11  away_shots_on_target  1140 non-null   int32         
 12  home_shots            1140 non-null   int32         
 13  away_shots        

In [11]:
#label match before and after covid19
epl_data.loc[epl_data['match_date']<='2020-03-10','covid19'] = 'before'
epl_data.loc[epl_data['match_date']>'2020-03-10','covid19'] = 'after'

epl_data

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR,home_win_rate,away_win_rate,covid19
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,0,33.33,10.00,0.17,0.00,0.50,0.00,1.00,0.00,before
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,3,40.00,83.33,0.00,0.17,0.00,0.20,0.00,1.00,before
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,3,16.67,30.77,0.00,0.23,0.00,0.75,0.00,1.00,before
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,0,75.00,30.77,0.25,0.08,0.33,0.25,1.00,0.00,before
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,3,13.33,33.33,0.07,0.13,0.50,0.40,0.00,1.00,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,0,52.38,37.50,0.24,0.00,0.45,0.00,1.00,0.00,after
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,0,25.00,30.00,0.08,0.00,0.33,0.00,1.00,0.00,after
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,0,50.00,29.41,0.21,0.00,0.43,0.00,1.00,0.00,after
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,3,28.57,44.44,0.07,0.22,0.25,0.50,0.00,1.00,after


## Data Analysis

The changes in stats are evaluated in 3 categories : 
* If the stats increases 2 % or more, it is categorized as 'positive effect'
* If the stats decreases -2 % or more, it is categorized as 'negative effect'
* If the stats changes within range -2% to 2%, it is categorized as 'relatively not affected'

In [47]:
def evaluate_stats(df,pos_chg=True,pct=True):
    #evaluate changes 
    ev = df.copy()

    if pos_chg == True:
    
        ev.loc[ev['changes_%']>=2,'effect'] = 'positive effect'  
        ev.loc[ev['changes_%']<=-2,'effect'] = 'negative effect'   
        ev.loc[(ev['changes_%']<2)&(ev['changes_%']>-2),'effect'] = 'relatively not affected'  
    
    else:
        
        ev.loc[ev['changes_%']>=2,'effect'] = 'negative effect'  
        ev.loc[ev['changes_%']<=-2,'effect'] = 'positive effect'   
        ev.loc[(ev['changes_%']<2)&(ev['changes_%']>-2),'effect'] = 'relatively not affected'
            
    evaluation = ev.value_counts('effect').to_frame(name='count')
    return evaluation

In [48]:
def get_home_stats(stats_name,df=epl_data,team=filtered_teams,evaluate=True,positive_changes=True):
    
    print('The effect of covid-19 on EPL home teams ' + stats_name)
    
    home_stats = pd.DataFrame()
    
    #home stats before covid-19
    home_stats['home_'+stats_name+'_before'] = \
        df.loc[df['covid19']=='before'].groupby('home_team')['home_'+stats_name].mean()
    
    #home stats after covid-19
    home_stats['home_'+stats_name+'_after'] = \
        df.loc[df['covid19']=='after'].groupby('home_team')['home_'+stats_name].mean()
    
    #filter team 
    home_stats = home_stats.loc[team]
    
    #changes in home stats
    home_stats['stats_changes'] = home_stats['home_'+stats_name+'_after'] - home_stats['home_'+stats_name+'_before']
    home_stats['changes_%'] = (home_stats['stats_changes']/home_stats['home_'+stats_name+'_before'])*100
    
    #sort by biggest change ascending
    home_stats = home_stats.sort_values('changes_%', ascending=True)
    
    if evaluate == True:
        display(evaluate_stats(home_stats,pos_chg=positive_changes))
    
    return home_stats

In [49]:
def get_away_stats(stats_name,df=epl_data,team=filtered_teams,evaluate=True,percentage=True,positive_changes=True):
    
    print('The effect of covid-19 on EPL away teams ' + stats_name)
    
    away_stats = pd.DataFrame()
    
    #away stats before covid-19
    away_stats['away_'+stats_name+'_before'] = \
        df.loc[df['covid19']=='before'].groupby('away_team')['away_'+stats_name].mean()
    
    #away stats after covid-19
    away_stats['away_'+stats_name+'_after'] = \
        df.loc[df['covid19']=='after'].groupby('away_team')['away_'+stats_name].mean()
    
    #filter team 
    away_stats = away_stats.loc[team]
    
    #changes in home stats
    away_stats['stats_changes'] = away_stats['away_'+stats_name+'_after'] - away_stats['away_'+stats_name+'_before']
    away_stats['changes_%'] = (away_stats['stats_changes']/away_stats['away_'+stats_name+'_before'])*100
    
    #sort by biggest change ascending
    away_stats = away_stats.sort_values('changes_%', ascending=True)
    
    if evaluate == True:
        display(evaluate_stats(away_stats,pos_chg=positive_changes))
    
    return away_stats

### Filter EPL Teams

The analysis will only observe the teams which had participated in all 3 seasons.

In [50]:
#get teams name for every season since 2018/2019 
teams_18_19 = set(epl_data.loc[epl_data['season']=='2018/2019']['home_team'].unique())
teams_19_20 = set(epl_data.loc[epl_data['season']=='2019/2020']['home_team'].unique())
teams_20_21 = set(epl_data.loc[epl_data['season']=='2020/2021']['home_team'].unique())

#filter the team that had participated in all 3 seasons
filtered_teams = list(teams_18_19.intersection(teams_19_20,teams_20_21))
filtered_teams

['Manchester United',
 'Burnley',
 'Chelsea',
 'Arsenal',
 'Crystal Palace',
 'Southampton',
 'Brighton and Hove Albion',
 'Wolverhampton Wanderers',
 'West Ham United',
 'Leicester City',
 'Everton',
 'Manchester City',
 'Liverpool',
 'Newcastle United',
 'Tottenham Hotspur']

### Home / Away Ball Possession %

In [51]:
get_home_stats('possession')

The effect of covid-19 on EPL home teams possession


,count
effect,
negative effect,6
positive effect,6
relatively not affected,3


,home_possession_before,home_possession_after,stats_changes,changes_%
home_team,,,,
Tottenham Hotspur,58.73,50.82,-7.91,-13.47
Arsenal,59.00,52.74,-6.26,-10.60
Crystal Palace,45.98,41.80,-4.18,-9.09
Everton,52.55,47.93,-4.63,-8.80
West Ham United,47.63,43.60,-4.03,-8.45
Chelsea,62.71,60.49,-2.21,-3.53
Manchester City,67.76,66.72,-1.04,-1.54
Leicester City,55.70,55.65,-0.04,-0.08
Newcastle United,41.60,41.93,0.33,0.80


In [52]:
get_away_stats('possession')

The effect of covid-19 on EPL away teams possession


,count
effect,
positive effect,7
negative effect,6
relatively not affected,2


,away_possession_before,away_possession_after,stats_changes,changes_%
away_team,,,,
Everton,49.26,45.14,-4.12,-8.37
Crystal Palace,43.43,40.45,-2.98,-6.86
West Ham United,45.55,42.45,-3.10,-6.81
Manchester City,66.74,62.74,-4.00,-6.00
Arsenal,55.41,52.71,-2.69,-4.86
Tottenham Hotspur,54.25,51.94,-2.31,-4.26
Liverpool,62.07,61.16,-0.91,-1.46
Brighton and Hove Albion,46.15,46.90,0.75,1.62
Newcastle United,35.99,36.77,0.78,2.18


### Home/Away Shots on Target

In [58]:
get_home_stats('shots_on_target')

The effect of covid-19 on EPL home teams shots_on_target


,count
effect,
negative effect,9
positive effect,5
relatively not affected,1


,home_shots_on_target_before,home_shots_on_target_after,stats_changes,changes_%
home_team,,,,
Everton,4.94,3.79,-1.15,-23.24
Burnley,4.00,3.26,-0.74,-18.48
West Ham United,4.88,4.04,-0.84,-17.16
Crystal Palace,3.82,3.26,-0.56,-14.72
Manchester United,6.47,5.70,-0.77,-11.98
Arsenal,4.88,4.30,-0.58,-11.84
Liverpool,6.71,5.91,-0.79,-11.82
Manchester City,7.38,6.68,-0.70,-9.42
Leicester City,5.06,4.78,-0.28,-5.46


In [59]:
get_away_stats('shots_on_target')

The effect of covid-19 on EPL away teams shots_on_target


,count
effect,
positive effect,8
negative effect,6
relatively not affected,1


,away_shots_on_target_before,away_shots_on_target_after,stats_changes,changes_%
away_team,,,,
Tottenham Hotspur,4.59,3.57,-1.02,-22.30
Manchester City,6.35,5.43,-0.92,-14.45
Wolverhampton Wanderers,3.88,3.33,-0.55,-14.06
Everton,4.15,3.65,-0.49,-11.93
Brighton and Hove Albion,3.50,3.17,-0.33,-9.32
Leicester City,5.03,4.67,-0.36,-7.23
Crystal Palace,3.39,3.46,0.06,1.90
Southampton,4.36,4.46,0.09,2.17
Liverpool,5.27,5.54,0.27,5.10


### Home/Away Shots

In [60]:
get_home_stats('shots')

The effect of covid-19 on EPL home teams shots


,count
effect,
negative effect,13
positive effect,1
relatively not affected,1


,home_shots_before,home_shots_after,stats_changes,changes_%
home_team,,,,
Crystal Palace,13.79,9.48,-4.32,-31.29
Everton,14.61,11.08,-3.52,-24.12
Tottenham Hotspur,15.00,12.17,-2.83,-18.89
Manchester City,20.81,17.68,-3.13,-15.05
Leicester City,14.94,13.00,-1.94,-12.99
Chelsea,17.29,15.30,-1.99,-11.51
Burnley,11.50,10.30,-1.20,-10.40
Newcastle United,13.36,12.21,-1.16,-8.65
Manchester United,15.62,14.43,-1.18,-7.57


In [61]:
get_away_stats('shots')

The effect of covid-19 on EPL away teams shots


,count
effect,
negative effect,7
positive effect,5
relatively not affected,3


,away_shots_before,away_shots_after,stats_changes,changes_%
away_team,,,,
Everton,11.85,9.39,-2.46,-20.77
Wolverhampton Wanderers,11.79,9.88,-1.91,-16.23
Southampton,12.45,11.21,-1.25,-10.01
Tottenham Hotspur,11.59,10.57,-1.02,-8.83
Manchester City,16.41,15.70,-0.72,-4.36
Chelsea,15.15,14.50,-0.65,-4.30
Crystal Palace,9.42,9.17,-0.26,-2.73
Newcastle United,8.97,8.83,-0.14,-1.61
Brighton and Hove Albion,10.26,10.22,-0.05,-0.46


### Home/Away Goals Scored per Game

In [63]:
get_home_stats('score')

The effect of covid-19 on EPL home teams score


,count
effect,
negative effect,9
positive effect,5
relatively not affected,1


,home_score_before,home_score_after,stats_changes,changes_%
home_team,,,,
Burnley,1.29,0.78,-0.51,-39.53
Liverpool,2.79,1.78,-1.01,-36.20
Arsenal,2.00,1.48,-0.52,-26.09
Everton,1.48,1.21,-0.28,-18.62
Wolverhampton Wanderers,1.44,1.17,-0.27,-18.54
Chelsea,1.79,1.70,-0.10,-5.49
Manchester City,2.81,2.68,-0.13,-4.71
Brighton and Hove Albion,1.09,1.04,-0.05,-4.51
West Ham United,1.67,1.62,-0.04,-2.50


In [64]:
get_away_stats('score')

The effect of covid-19 on EPL away teams score


,count
effect,
positive effect,8
negative effect,6
relatively not affected,1


,away_score_before,away_score_after,stats_changes,changes_%
away_team,,,,
Wolverhampton Wanderers,1.18,0.79,-0.39,-33.01
Crystal Palace,1.39,0.96,-0.44,-31.25
Everton,1.24,1.09,-0.15,-12.01
Burnley,1.06,1.00,-0.06,-5.71
Leicester City,1.67,1.58,-0.08,-5.00
Chelsea,1.61,1.54,-0.06,-4.01
Manchester City,2.15,2.17,0.03,1.25
Brighton and Hove Albion,0.91,0.96,0.04,4.91
Arsenal,1.41,1.48,0.07,5.24


### Home/Away Win Rate

In [65]:
get_home_stats('win_rate')

The effect of covid-19 on EPL home teams win_rate


,count
effect,
negative effect,11
positive effect,2
relatively not affected,2


,home_win_rate_before,home_win_rate_after,stats_changes,changes_%
home_team,,,,
Burnley,0.41,0.22,-0.19,-47.20
Everton,0.52,0.29,-0.22,-43.38
Liverpool,0.94,0.57,-0.38,-39.95
Brighton and Hove Albion,0.30,0.21,-0.09,-31.25
Newcastle United,0.39,0.29,-0.10,-25.96
Arsenal,0.62,0.48,-0.14,-22.57
Crystal Palace,0.32,0.26,-0.06,-19.37
Manchester City,0.84,0.76,-0.08,-9.93
Manchester United,0.53,0.48,-0.05,-9.66


In [66]:
get_away_stats('win_rate')

The effect of covid-19 on EPL away teams win_rate


,count
effect,
positive effect,10
negative effect,5


,away_win_rate_before,away_win_rate_after,stats_changes,changes_%
away_team,,,,
Liverpool,0.76,0.50,-0.26,-34.00
Crystal Palace,0.39,0.29,-0.10,-25.96
Wolverhampton Wanderers,0.33,0.29,-0.04,-12.50
Tottenham Hotspur,0.41,0.39,-0.02,-4.97
Southampton,0.30,0.29,-0.01,-3.75
Manchester City,0.68,0.70,0.02,2.84
Chelsea,0.48,0.50,0.02,3.12
Leicester City,0.42,0.46,0.03,8.04
Newcastle United,0.24,0.30,0.07,29.35


### Home/Away Points per Game

In [67]:
get_home_stats('points')

The effect of covid-19 on EPL home teams points


,count
effect,
negative effect,11
positive effect,2
relatively not affected,2


,home_points_before,home_points_after,stats_changes,changes_%
home_team,,,,
Liverpool,2.88,1.87,-1.01,-35.14
Everton,1.79,1.17,-0.62,-34.75
Burnley,1.35,1.00,-0.35,-26.09
Arsenal,2.09,1.65,-0.44,-20.88
Newcastle United,1.39,1.17,-0.23,-16.30
Brighton and Hove Albion,1.24,1.04,-0.20,-16.16
Crystal Palace,1.24,1.04,-0.19,-15.53
Manchester United,1.91,1.70,-0.22,-11.30
Wolverhampton Wanderers,1.65,1.48,-0.17,-10.25


In [68]:
get_away_stats('points')

The effect of covid-19 on EPL away teams points


,count
effect,
positive effect,11
negative effect,3
relatively not affected,1


,away_points_before,away_points_after,stats_changes,changes_%
away_team,,,,
Crystal Palace,1.39,1.00,-0.39,-28.26
Liverpool,2.45,1.79,-0.66,-27.01
Wolverhampton Wanderers,1.33,1.12,-0.21,-15.62
Chelsea,1.64,1.67,0.03,1.85
Southampton,1.09,1.12,0.03,3.13
Tottenham Hotspur,1.41,1.48,0.07,4.71
Manchester City,2.12,2.22,0.10,4.71
Newcastle United,1.00,1.13,0.13,13.04
Leicester City,1.45,1.67,0.21,14.58


### Home/Away Shots on Target % 

In [70]:
get_home_stats('SoT%')

The effect of covid-19 on EPL home teams SoT%


,count
effect,
positive effect,10
negative effect,4
relatively not affected,1


,home_SoT%_before,home_SoT%_after,stats_changes,changes_%
home_team,,,,
Manchester United,44.37,37.55,-6.82,-15.36
Burnley,34.64,31.61,-3.03,-8.74
Liverpool,39.05,37.29,-1.76,-4.51
West Ham United,39.46,37.92,-1.54,-3.90
Everton,34.06,33.68,-0.38,-1.13
Brighton and Hove Albion,27.46,28.17,0.71,2.58
Leicester City,35.16,36.32,1.16,3.29
Arsenal,37.01,39.07,2.06,5.56
Wolverhampton Wanderers,31.02,33.19,2.17,7.00


### Home/Away Quantity Conversion Rate

In [71]:
get_home_stats('quan_CR')

The effect of covid-19 on EPL home teams quan_CR


,count
effect,
positive effect,10
negative effect,5


,home_quan_CR_before,home_quan_CR_after,stats_changes,changes_%
home_team,,,,
Burnley,0.12,0.08,-0.04,-31.96
Wolverhampton Wanderers,0.12,0.09,-0.04,-30.77
Liverpool,0.17,0.13,-0.04,-24.20
Brighton and Hove Albion,0.11,0.08,-0.02,-19.67
Arsenal,0.15,0.14,-0.01,-6.58
Chelsea,0.11,0.11,0.00,4.13
Manchester United,0.13,0.14,0.01,6.52
Leicester City,0.12,0.13,0.01,8.80
Manchester City,0.14,0.16,0.02,13.75


In [72]:
get_away_stats('quan_CR')

The effect of covid-19 on EPL away teams quan_CR


,count
effect,
positive effect,9
negative effect,5
relatively not affected,1


,away_quan_CR_before,away_quan_CR_after,stats_changes,changes_%
away_team,,,,
Burnley,0.16,0.09,-0.07,-43.88
Wolverhampton Wanderers,0.12,0.08,-0.04,-35.25
Crystal Palace,0.17,0.11,-0.06,-32.98
Arsenal,0.16,0.14,-0.02,-12.75
Liverpool,0.14,0.13,-0.02,-10.75
Chelsea,0.11,0.11,0.00,0.81
Newcastle United,0.10,0.10,0.00,2.68
Tottenham Hotspur,0.15,0.15,0.01,3.55
Brighton and Hove Albion,0.10,0.10,0.01,6.11


### Home/Away Quality Conversion Rate

In [73]:
get_home_stats('qual_CR')

The effect of covid-19 on EPL home teams qual_CR


,count
effect,
positive effect,8
negative effect,6
relatively not affected,1


,home_qual_CR_before,home_qual_CR_after,stats_changes,changes_%
home_team,,,,
Burnley,0.40,0.27,-0.14,-33.53
Liverpool,0.43,0.29,-0.14,-32.53
Wolverhampton Wanderers,0.38,0.28,-0.11,-27.56
Arsenal,0.42,0.33,-0.10,-22.81
Brighton and Hove Albion,0.35,0.27,-0.08,-21.74
Chelsea,0.32,0.27,-0.05,-15.84
Southampton,0.30,0.30,-0.00,-0.22
Leicester City,0.33,0.34,0.01,3.22
Manchester City,0.38,0.41,0.02,6.49


In [74]:
get_away_stats('qual_CR')

The effect of covid-19 on EPL away teams qual_CR


,count
effect,
positive effect,8
negative effect,7


,away_qual_CR_before,away_qual_CR_after,stats_changes,changes_%
away_team,,,,
Burnley,0.46,0.25,-0.21,-45.06
Wolverhampton Wanderers,0.34,0.24,-0.10,-29.53
Crystal Palace,0.44,0.33,-0.11,-25.33
Newcastle United,0.34,0.29,-0.05,-13.95
Chelsea,0.35,0.31,-0.04,-12.72
Arsenal,0.41,0.38,-0.03,-7.93
Liverpool,0.35,0.33,-0.02,-5.51
Leicester City,0.34,0.35,0.01,3.28
Everton,0.28,0.30,0.02,6.40
